In [49]:
import os
import random
from typing import Optional


import datasets
import groq

In [10]:
class GroqInterface:
    _client = None 

    LLAMA3_70B = "llama3-70b-8192"

    def __init__(self, model:Optional[str]=None):
        if GroqInterface._client is None:
            api_key = os.environ.get("GROQ_API_KEY")

            if api_key is None:
                raise RuntimeError("API key is not in the environment variables ('GROQ_API_KEY' variable is not set).")

            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_70B
        self._model = model

    def __call__(self, prompt:str):
        chat_completion = GroqInterface._client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=self._model,
            )

        return chat_completion.choices[0].message.content

In [11]:
groq_interface = GroqInterface()

In [13]:
groq_interface("Hi!")

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [48]:
POSITIVE = 1
NEGATIVE = 0

In [51]:
class GroqSentimentInterface(GroqInterface):

    def __call__(self, prompt: str):
        response = super().__call__(prompt)
        response = response.lower()

        if "positive" in response and "negative" not in response:
            return POSITIVE
        if "negative" in response and "positive" not in response:
            return NEGATIVE
        
        return random.choice([POSITIVE, NEGATIVE])

In [52]:
groq_sentiment = GroqSentimentInterface()

## IMDB Prompt Engineering

In [16]:
trainbase_dataset = datasets.load_dataset('imdb', split='train')
test_dataset = datasets.load_dataset('imdb', split='test')

train_val_dataset = trainbase_dataset.train_test_split(test_size=0.2)

train_dataset = train_val_dataset["train"]
val_dataset = train_val_dataset["test"]

In [55]:
base_prompt = '''Classify if the movie review is positive or negative: 
                Review:
                Movie review

                Sentiment:
                ONLY POSITIVE OR NEGATIVE

                Classify if this movie review is positive or negative:
                Review:
                {example1}

                Sentiment:
                {response1}

                Classify if this movie review is positive or negative:
                Review:
                {review}
                
                Sentiment:
                
                '''

In [56]:
label = "POSITIVE" if train_dataset[0]["label"]==POSITIVE else "NEGATIVE"
prompt = base_prompt.format(example1=train_dataset[0]["text"], response1=label, review=train_dataset[1]["text"])

In [57]:
groq_sentiment(prompt)

0